<a href="https://colab.research.google.com/github/sujantkumarkv/extend_context/blob/main/extend_gemma_ctx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip3 install -q kagglehub torch sentencepiece

In [6]:
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


## Install dependencies

In [3]:
# !pip3 install -q torch immutabledict sentencepiece # datasets

## Download model weights

In [7]:
# Choose variant and machine type
VARIANT = '2b-it' #@param ['2b', '2b-it', '7b', '7b-it', '7b-quant', '7b-it-quant']
MACHINE_TYPE = 'cuda' #@param ['cuda', 'cpu']

In [9]:
import os

# Load model weights
weights_dir = kagglehub.model_download(f'google/gemma/pyTorch/{VARIANT}')

# Ensure that the tokenizer is present
tokenizer_path = os.path.join(weights_dir, 'tokenizer.model')
# assert os.path.isfile(tokenizer_path), 'Tokenizer not found!'

# Ensure that the checkpoint is present
ckpt_path = os.path.join(weights_dir, f'gemma-{VARIANT}.ckpt')
# assert os.path.isfile(ckpt_path), 'PyTorch checkpoint not found!'

In [21]:
print(ckpt_path)
print(weights_dir)

/teamspace/studios/this_studio/.cache/kagglehub/models/google/gemma/pyTorch/2b-it/2/gemma-2b-it.ckpt
/teamspace/studios/this_studio/.cache/kagglehub/models/google/gemma/pyTorch/2b-it/2


In [30]:
!ls -a /teamspace/studios/this_studio/.cache/kagglehub/models/google/gemma/pyTorch/2b-it/2

.  ..  config.json  gemma-2b-it.ckpt  tokenizer.model


## model implementation

In [6]:
# NOTE: The "installation" is just cloning the repo.
!git clone https://github.com/google/gemma_pytorch.git

fatal: destination path 'gemma_pytorch' already exists and is not an empty directory.


⚡️ Tip	Check organization access: https://github.com/settings/connections/applications/c7457225b242a94d60c6



In [23]:
import sys
sys.path.append('gemma_pytorch')

In [24]:
from gemma_pytorch.gemma.config import get_config_for_2b
from gemma_pytorch.gemma.model import GemmaForCausalLM

## Setup the model

In [25]:
import torch

# Set up model config.
model_config = get_config_for_2b()
model_config.tokenizer = tokenizer_path
model_config.quant = 'quant' in VARIANT

# Instantiate the model and load the weights.
torch.set_default_dtype(model_config.get_dtype())
device = torch.device(MACHINE_TYPE)
model = GemmaForCausalLM(model_config)
model.load_weights(ckpt_path)
model = model.to(device).eval()

In [ ]:
!wget https://raw.githubusercontent.com/sujantkumarkv/extend_context/main/rocstories.txt

--2024-06-18 15:14:36--  https://raw.githubusercontent.com/sujantkumarkv/extend_context/main/rocstories.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 18007969 (17M) [text/plain]
Saving to: ‘rocstories.txt.1’

rocstories.txt.1    100%[===================>]  17.17M  --.-KB/s    in 0.1s    

2024-06-18 15:14:37 (139 MB/s) - ‘rocstories.txt.1’ saved [18007969/18007969]



In [26]:
with open("rocstories.txt", 'r') as text_input:
    text = text_input.read()

In [27]:
from gemma_pytorch.gemma.tokenizer import Tokenizer

tokenizer = Tokenizer(tokenizer_path)
# len(tokenizer.encode(text)) # 314936
tokens4k = tokenizer.encode(text)[: 4000]
tokens8k = tokenizer.encode(text)[: 8000]
tokens16k = tokenizer.encode(text)[: 16000]
# tokens32k = tokenizer.encode(text)[: 32768]

text4k = tokenizer.decode(tokens4k)
text8k = tokenizer.decode(tokens8k)
text16k = tokenizer.decode(tokens16k)

In [13]:
len(text), len(tokenizer.encode(text)), len(tokens8k), len(tokens16k)

(18007898, 4086816, 8000, 16000)

In [28]:
tokensprompt_4k = tokenizer.decode(tokens4k[800: 1000])
tokensprompt_6k = tokenizer.decode(tokens8k[4800: 5000])
tokensprompt_8k = tokenizer.decode(tokens8k[7800: 7900])

print(f'tokensprompt_4k\n {tokensprompt_4k}\n')
print(f'tokensprompt_6k\n {tokensprompt_6k}')
print(f'tokensprompt_8k\n {tokensprompt_8k}')

tokensprompt_4k
 Al was taking a class for college. He needed a certain book for his class. He was talking to his friend about finding the book. His friend told him he already had the book. Al's friend let him borrow the book for class.
Trudy felt sad so thought she needed a pet. She decided to buy a Puli. She called some Puli owners in town and asked to buy one. The third Puli owner agreed to sell Trudy one of her dogs. So Trudy bought a Puli and is now much happier.
Max noticed he was always tired so he went to the doctor for help. His doctor asked Max if he slept through the night regularly. Max couldn't remember the last time he had slept through the night. The doctor gave him a small sample of sleeping pills to try. After taking a pill for four nights, Max felt awake during the day.
Joshua couldn't afford the assigned textbooks. His college textbooks were going to cost over 50

tokensprompt_6k
  having a blast. It was summer break and they loved playing outside. Their favorite gam

In [26]:
# # change basae RoPE theta
# from gemma_pytorch.gemma.config import GemmaConfig

# config = GemmaConfig()
# config.rope_theta = 100000000

In [29]:
USER_CHAT_TEMPLATE = '<start_of_turn>user\n{prompt}<end_of_turn>\n'
MODEL_CHAT_TEMPLATE = '<start_of_turn>model\n{prompt}<end_of_turn>\n'

prompt = (
    USER_CHAT_TEMPLATE.format(
        prompt=f'Read the following properly and answer questions based on it. \n {text4k}'
    )
    + MODEL_CHAT_TEMPLATE.format(prompt='Yes, sure I can help with that.')
    + USER_CHAT_TEMPLATE.format(prompt='Trudy felt sad so what did she thought of?')
    + '<start_of_turn>model\n'
)
# print('Chat prompt:\n', prompt)

model.generate(
    prompts=prompt,
    device=device,
    output_len=100,
)

'Trudy felt sad when she heard that she had been told that she had sold the most. She was disappointed that she lost out on the chance to win the radio.'

## FINETUNE

In [31]:
!pip3 install torch=="2.1.2"
!pip3 install -e git+https://github.com/OpenAccess-AI-Collective/axolotl#egg=axolotl
!pip3 install flash-attn=="2.5.0"
!pip3 install deepspeed=="0.13.1"
!pip3 install mlflow=="2.13.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 7.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 20.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 41.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: triton
    Found existing installation: triton 2.2.0
    Uninstalling triton-2.2.0:
      Successfully uninstalled triton-2.2.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.19.3
    Uninstalling nvidia-nccl-cu12-2.19.3:
      Successfully uninstalled nvidia-nccl-cu12-2.19.3
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1+cu121
    Uninstalling torch-2.2.1+cu121:
      Successfully uninstalled torch-2.2.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.17.1+cu121 requires torch=

In [32]:
import yaml

yaml_string = """
base_model: mhenrichsen/gemma-7b
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer

load_in_8bit: true
load_in_4bit: false
strict: false

# huggingface repo
datasets:
  - path: mhenrichsen/alpaca_2k_test
    type: alpaca
val_set_size: 0.1
output_dir: ./outputs/out

adapter: lora
lora_r: 32
lora_alpha: 16
lora_dropout: 0.05
lora_target_linear: true

sequence_len: 8192
sample_packing: true
eval_sample_packing: false
pad_to_sequence_len: true

wandb_project:
wandb_entity:
wandb_watch:
wandb_name:
wandb_log_model:


gradient_accumulation_steps: 3
micro_batch_size: 2
num_epochs: 4
optimizer: adamw_bnb_8bit
lr_scheduler: cosine
learning_rate: 0.0002

train_on_inputs: false
group_by_length: false
bf16: auto
fp16:
tf32: false

gradient_checkpointing: true
early_stopping_patience:
resume_from_checkpoint:
local_rank:
logging_steps: 1
xformers_attention:
flash_attention: true

warmup_ratio: 0.1
evals_per_epoch: 4
eval_table_size:
eval_max_new_tokens: 128
saves_per_epoch: 1
debug:
deepspeed:
weight_decay: 0.0
fsdp:
fsdp_config:
special_tokens:
"""

In [ ]:
yaml_dict = yaml.safe_load(yaml_string)

# Specify your file path
file_path = 'test_axolotl.yaml'

# Write the YAML file
with open(file_path, 'w') as file:
    yaml.dump(yaml_dict, file)

## EVAL

Naively changing RoPE theta & frequencies don't work too well. need to lora finetune maybe.